 Выполнить задания:
1.	Нормализовать базу данных относительно актеров. Выделить таблицу с актерами, а также таблицу многие-ко-многим `актер-фильм`. (Т.е. будут 3 таблицы: актеры, фильмы, актер-фильм. Сохранить новую бд как ‘task1.sqlite’.
2.	Исходя из этой таблицы, средствами SQL, вычислить наболее часто работающую друг с другом пару актеров.


Добавим библиотеки

In [20]:
import sqlite3
import re

Создадим базу данных и назовем ее 'task1.sqlite'
 
Создадим курсор, который используется для выполнения операций базы данных, таких как выполнение SQL-запросов и извлечение данных.

In [21]:
conn = sqlite3.connect('task1.sqlite') 
cursor = conn.cursor()

Создадим таблицы actors (для актеров), movies (фильмы), actor_movie (таблица-связь многие ко многим актер-фильм)

In [ ]:

cursor.execute('''CREATE TABLE actors (
                id INTEGER PRIMARY KEY,
                name TEXT )
''')


In [ ]:

cursor.execute('''CREATE TABLE movies (
                id INTEGER PRIMARY KEY,
                title TEXT )
''')

cursor.execute('''CREATE TABLE actor_movie ( 
               actor_id INTEGER, 
               movie_id INTEGER, 
               FOREIGN KEY (actor_id) REFERENCES actors(id), 
               FOREIGN KEY (movie_id) REFERENCES movies(id) )
''')

conn.commit()
conn.close()

Достаем информацию

In [24]:

connect_netflix = sqlite3.connect('netflix.sqlite')

cursor_netflix = connect_netflix.cursor()

cursor_netflix.execute('select title, "cast" from netflix_titles')

data = cursor_netflix.fetchall()

connect_netflix.close()

Преобразуем в удобный вид

In [25]:

data_new = []

uniquely_actors = set()
.
for line in data:
    movie, cast_str = line

    cast = re.split(r' ,|, |,', cast_str)

    uniquely_actors = uniquely_actors.union(set(cast))

    data_new.append((movie, cast))

Заполнение фильмами

In [26]:

connect_movies = sqlite3.connect('task1.sqlite')

cursor_movies = connect_movies.cursor()

for line in data_new:
    movie, actors = line
     
    cursor_movies.execute("INSERT INTO movies VALUES (NULL, ?)", (movie,))

connect_movies.commit()
connect_movies.close()

Заполение актерами

In [27]:

connect_actors = sqlite3.connect('task1.sqlite')
cursor_actors = connect_actors.cursor()

for actor in uniquel_actors:
    cursor_actors.execute("INSERT INTO actors VALUES (NULL, ?)", (actor,))

connect_actors.commit()
connect_actors.close()

Соединение актёр-фильм

In [28]:

connect_task1 = sqlite3.connect('task1.sqlite')

cursor_movies = connect_task1.cursor()
cursor_actors = connect_task1.cursor()
cursor_actor_movie = connect_task1.cursor()

for i in data_new:

    movie, actors = i
    cursor_movies.execute('SELECT id FROM movies WHERE title = ?', (movie,))
    movie_id = cursor_movies.fetchone()
    for actor in actors:
        cursor_actors.execute('SELECT id FROM actors WHERE name = ?', (actor,))
        actor_id = cursor_actors.fetchone()

        cursor_actor_movie.execute("INSERT INTO actor_movie (actor_id, movie_id) VALUES (?, ?)", (actor_id[0], movie_id[0])) 

connect_task1.commit()

connect_task1.close()

Поиск актеров 

Этот код выполняет запрос к базе данных SQLite, чтобы найти пару актеров, которые чаще всего снимались вместе.

In [29]:

conn = sqlite3.connect('task1.sqlite')

cursor = conn.cursor()

cursor.execute('''
    SELECT a1.name AS actor1_name, a2.name AS actor2_name, COUNT(*) AS movies_together
    FROM actor_movie am1
    JOIN actor_movie am2 ON am1.movie_id = am2.movie_id AND am1.actor_id < am2.actor_id
    JOIN actors a1 ON am1.actor_id = a1.id
    JOIN actors a2 ON am2.actor_id = a2.id
    GROUP BY actor1_name, actor2_name
    ORDER BY movies_together DESC
    LIMIT 1
''')

result = cursor.fetchone()

print("Самая часто работающая друг с другом пара актеров:")
print(f"{result[0]} и {result[1]} - сыграли вместе в {result[2]} фильмах")

conn.close()

Самая часто работающая друг с другом пара актеров:
John Paul Tremblay и Robb Wells - сыграли вместе в 15 фильмах
